In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 34.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.9 MB/s 
     |████████████████████████████████| 182 kB 73.6 MB/s 


In [3]:
from google.colab import drive
from transformers import BertTokenizer
import pandas as pd
import numpy as np
import torch
import pickle

drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

%pip install tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'1.12.1+cu113'

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [4]:
# load the model from disk
main_path = './drive/MyDrive/NLP Project Work/'
filename = f'{main_path}model.sav'
model = pickle.load(open(filename, 'rb'))

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [8]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [16]:
data = pd.read_csv(f'{main_path}eng_train.txt', encoding="utf-8", sep=" ", header=None, skip_blank_lines=False)

In [17]:
data.columns = ["Word", "POS", "IOB tags", "Tag"]

In [18]:
# add a new column "sentence", seperating rows by empty lines
i = 0
sentence_list = []
for row in data["Word"]:
  if isinstance(row, str):
    sentence_list.append(str(i))
  else:
    i += 1
    sentence_list.append(float('nan'))

In [19]:
data["Sentence"] = sentence_list
data = data.dropna()
data.head(10)

,Word,POS,IOB tags,Tag,Sentence
0,EU,NNP,I-NP,I-ORG,0
1,rejects,VBZ,I-VP,O,0
2,German,JJ,I-NP,I-MISC,0
3,call,NN,I-NP,O,0
4,to,TO,I-VP,O,0
5,boycott,VB,I-VP,O,0
6,British,JJ,I-NP,I-MISC,0
7,lamb,NN,I-NP,O,0
8,.,.,O,O,0
10,Peter,NNP,I-NP,I-PER,1


In [20]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""

In [12]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()

In [13]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [21]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [22]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

B-MISC	[CLS]
B-MISC	Mr
B-MISC	.
O	Trump
B-MISC	’
B-MISC	s
B-MISC	tweets
B-MISC	began
B-MISC	just
B-MISC	moments
B-MISC	after
B-MISC	a
B-ORG	Fox
B-ORG	News
B-MISC	report
B-MISC	by
O	Mike
O	Tobin
B-MISC	,
B-MISC	a
B-MISC	reporter
B-MISC	for
B-MISC	the
B-MISC	network
B-MISC	,
B-MISC	about
B-MISC	protests
B-MISC	in
I-LOC	Minnesota
B-MISC	and
B-MISC	elsewhere
B-MISC	.
B-MISC	[SEP]
